# Tagging - Interpretando dados com funções

Uma das principais aplicações das funções externas por desenvolvedores não é exatamente na utilização dessas funções e sim, na utilização desta sintaxe e estruturação de dados gerado pela API da OpenAI para a categorização e estruturação de dados em texto. Daremos um exemplo aqui ao categorizanto sentimentos de falas.

In [6]:
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

class Sentimento(BaseModel):
    '''Define o sentimento e a língua da mensagem enviada'''
    sentimento: str = Field(description='Sentimento da mensagem. Deve ser "positivo", "negativo" ou "não definido"')
    lingua: str = Field(description='Língua que a mensagem foi escrita. Deve estar no formato ISO 639-1')

tool_sentimento = convert_to_openai_function(Sentimento)
tool_sentimento

{'name': 'Sentimento',
 'description': 'Define o sentimento e a língua da mensagem enviada',
 'parameters': {'properties': {'sentimento': {'description': 'Sentimento da mensagem. Deve ser "positivo", "negativo" ou "não definido"',
    'type': 'string'},
   'lingua': {'description': 'Língua que a mensagem foi escrita. Deve estar no formato ISO 639-1',
    'type': 'string'}},
  'required': ['sentimento', 'lingua'],
  'type': 'object'}}

In [7]:
texto = 'Eu gosto muito de massa aos quatro queijos'

In [10]:
from langchain.prompts import ChatPromptTemplate as cpt
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

_=load_dotenv(find_dotenv())

prompt = cpt.from_messages([    
        ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
        ('user', '{input}')
    ])

chat = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | chat.bind(functions=[tool_sentimento], function_call={'name': 'Sentimento'})

In [14]:
chain.invoke({'input':'Eu gosto muito de Laranjas'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"positivo","lingua":"pt"}', 'name': 'Sentimento'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 122, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae51e9b7-ff32-43ed-a085-296c5bcffcdf-0', usage_metadata={'input_tokens': 122, 'output_tokens': 11, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
chain.invoke({'input': 'I dont like this place'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"negativo","lingua":"en"}', 'name': 'Sentimento'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 120, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e07dca0-ba4c-4faa-9e14-62d61001d646-0', usage_metadata={'input_tokens': 120, 'output_tokens': 12, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Parseando a saída para obtermos apenas o que interessa

Podemos utilizar o JsonOutputFunctionsParser para obtermos como resultado final da nossa chain apenas o que nos interessa, que é o dicionário com as tags do conteúdo.

In [18]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = prompt | chat.bind(functions=[tool_sentimento], function_call={'name': 'Sentimento'}) | JsonOutputFunctionsParser()

chain.invoke({'input': 'Eu gosto muito de pizza'})

{'sentimento': 'positivo', 'lingua': 'pt'}

In [19]:
chain.invoke({'input': 'I love my girlfriend'})

{'sentimento': 'positivo', 'lingua': 'en'}

## Um exemplo mais interessante

Indo para um exemplo um pouco mais complexo. Digamos que temos um chatbot e queremos fazer um roteamento para os setores de interesse, no nosso caso atendimento_cliente, duvidas_alunos, vendas, spam. A primeira etapa é criar um modelo que entenda a solicitação do cliente e direcione para o setor certo. Nas próximas etapas o atendimento pode ser continuado por pessoas ou por agentes especializados para as tarefas do setor.
Retiramos as seguintes mensagens de email recebidos pela nossa equipe de atendimento e vamos tentar criar um direcionamento para elas:

In [21]:
duvidas = [
    'Bom dia, gostaria de saber se há um certificado final para cada trilha ou se os certificados são somente para os cursos e projetos? Obrigado!',
    'In Etsy, Amazon, eBay, Shopify https://pint77.com Pinterest+SEO +II = high sales results',
    'Boa tarde, estou iniciando hoje e estou perdido. Tenho vários objetivos. Não sei nada programação, exceto que utilizo o Power automate desktop da Microsoft. Quero aprender tudo na plataforma que se relacione ao Trading de criptomoedas. Quero automatizar Tradings, fazer o sistema reconhecer padrões, comprar e vender segundo critérios que eu defina, etc. Também tenho objetivos de aprender o máximo para utilizar em automações no trabalho também, que envolve a área jurídica e trabalho em processos. Como sou fã de eletrônica e tenho cursos na área, também queria aprender o que precisa para automatizacões diversas. Existe algum curso ou trilha que me prepare com base para todas essas áreas ao mesmo tempo e a partir dele eu aprenda isoladamente aquilo que seria exigido para aplicar aos meus projetos?',
    'Bom dia, Havia pedido cancelamento de minha mensalidade no mes 2 e continuaram cobrando. Peço cancelamento da assinatura. Peço por gentileza, para efetivarem o cancelamento da assomatura e pagamento.',
    'Bom dia. Não estou conseguindo tirar os certificados dos cursos que concluí. Por exemplo, já consegui 100% no python starter, porém, não consigo tirar o certificado. Como faço?',
    'Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?'
    ]

In [22]:
from enum import Enum
from pydantic import BaseModel, Field

class SetorEnum(str, Enum):
    atendimento_cliente = 'atendimento_cliente'
    duvidas_aluno = 'duvidas_aluno'
    vendas = 'vendas'
    spam = 'spam'

class DirecionaSetorResponsavel(BaseModel):
    '''Direciona a dúvida de um cliente ou aluno da escola de programação Asimov para o setor responsável'''
    setor : SetorEnum

In [27]:
from dotenv import load_dotenv, find_dotenv
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate as cpt
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
_=load_dotenv(find_dotenv())

tool_setor = convert_to_openai_function(DirecionaSetorResponsavel)

prompt = cpt.from_messages([
    ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
chat_func = chat.bind(functions=[tool_setor], function_call={'name': 'DirecionaSetorResponsavel'})

chain = prompt | chat_func | JsonOutputFunctionsParser()

In [29]:
duvida = duvidas[0]
resposta = chain.invoke({'input': duvida})
print('Dúvida ', duvida)
print('Resposta ', resposta)

Dúvida  Bom dia, gostaria de saber se há um certificado final para cada trilha ou se os certificados são somente para os cursos e projetos? Obrigado!
Resposta  {'setor': 'duvidas_aluno'}


In [32]:
duvida = duvidas[5]
resposta = chain.invoke({'input': duvida})
print('Dúvida ', duvida)
print('Resposta ', resposta)

Dúvida  Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?
Resposta  {'setor': 'vendas'}


Neste caso, gostaríamos que a dúvida fosse direcionada para vendas. Podemos melhorar nosso prompt para auemntar a chance do modelo responder como gostaríamos:

In [ ]:
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a dúvidas de preço, sobre o produto, como funciona devem ser direciodas para "vendas".
Questões relacionadas a conta, acesso a plataforma, a cancelamento e renovação de assinatura para devem ser direciodas para "atendimento_cliente".
Questões relacionadas a dúvidas técnicas de programação, conteúdos da plataforma ou tecnologias na área da programação devem ser direciodas para "duvidas_alunos".
Mensagens suspeitas, em outras línguas que não português, contendo links devem ser direciodas para "spam".
'''


In [33]:
from dotenv import load_dotenv, find_dotenv
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate as cpt
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
_=load_dotenv(find_dotenv())

tool_setor = convert_to_openai_function(DirecionaSetorResponsavel)

prompt = cpt.from_messages([
    ('system', '''Pense com cuidado ao categorizar o texto conforme as instruções.
    Questões relacionadas a dúvidas de preço, sobre o produto, como funciona devem ser direciodas para "vendas".
    Questões relacionadas a conta, acesso a plataforma, a cancelamento e renovação de assinatura para devem ser direciodas para "atendimento_cliente".
    Questões relacionadas a dúvidas técnicas de programação, conteúdos da plataforma ou tecnologias na área da programação devem ser direciodas para "duvidas_alunos".
    Mensagens suspeitas, em outras línguas que não português, contendo links devem ser direciodas para "spam".
    '''),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
chat_func = chat.bind(functions=[tool_setor], function_call={'name': 'DirecionaSetorResponsavel'})

chain = prompt | chat_func | JsonOutputFunctionsParser()

In [34]:
duvida = duvidas[5]
resposta = chain.invoke({'input': duvida})
print('Dúvida ', duvida)
print('Resposta ', resposta)

Dúvida  Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?
Resposta  {'setor': 'vendas'}
